In [2]:
import pandas as pd
import tqdm
import datetime
from datetime import timedelta
import os

# Učitavanje originalnih podataka 

In [3]:
dataset_path = '../data/raw/export_df.parquet'

In [4]:
data = pd.read_parquet(dataset_path)
data.head()


,hotel_id,datum_dolaska,datum_kreiranja_rezervacije,datum_odjave,datum_otkazivanja_rezervacije,broj_odraslih_gostiju,broj_djece_gostiju,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,status_rezervacije,rezervacija_id,gost_id
0,0,2015-07-01,2015-01-21,2015-07-01,None,2,0.0,PRT,0,0,100.000000,Check-Out,1313223,1077152
1,0,2015-07-01,2015-06-10,2015-07-01,None,2,0.0,PRT,0,0,100.000000,Check-Out,1313224,1017906
2,0,2015-07-01,2015-05-13,2015-07-02,None,1,0.0,GBR,0,1,64.991345,Check-Out,1313225,1039896
3,0,2015-07-01,2014-05-30,2015-07-02,None,1,0.0,GBR,1,1,74.368897,Check-Out,1313226,1008245
4,0,2015-07-01,2014-07-06,2015-07-03,None,2,0.0,GBR,2,1,130.973278,Check-Out,1313227,1093703


#### Uklanjanje zapisa (redaka) gdje je zemlja gosta 0 

In [5]:
data= data[~(data['zemlja_gosta']=='0')]

#### Dodavanje stupca  ukupno_gostiju ; broj djece za svaki redak (djeca + odrasli)

In [6]:
data['ukupno_gostiju'] = data['broj_odraslih_gostiju'] + data['broj_djece_gostiju']
data['ukupno_gostiju'] = data['ukupno_gostiju'].astype('int')

#### Uklanjanje rezervacija u kojima nema gostiju, odnosno ukupan je broj gostiju 0

NOTE: Zašto ove retke uklanjamo sada, a ne nakon primjene statističke ili metode strojnog učenja? Jer ovdje govorimo o retcima u kojima je broj gostiju nula, što znači kako su ti podaci nevaljani: označen je dolazak gosta, odnosno očekujemo da je gost došao, ali je zbog nekog razloga krivo postavljen broj gostiju te možemo reći da su ovdje krivo uneseni podaci

In [7]:
data= data[data['ukupno_gostiju'] != 0]

#### Dodavanje stupca duljina boravka

In [8]:
data['duljina_boravka'] = data['datum_odjave'] - data['datum_dolaska']
data['duljina_boravka'] = data['duljina_boravka'].dt.days

#### Uklanjanje redaka gdje je duljina boravka 0

In [9]:
data = data[data['duljina_boravka']>0]

#### Dodavanje stupca raspon_dolazak_rezervacija

In [10]:
data['raspon_dolazak_rezervacija'] = (data['datum_dolaska'] - data['datum_kreiranja_rezervacije']).dt.days

#### Uklanjanje (u zaseban df) radaka s otkazanim rezervacijama

In [11]:
data_canceled = data[data['datum_otkazivanja_rezervacije'].notna()]
data_canceled = data_canceled.reset_index(drop=True)
data = data[~data['datum_otkazivanja_rezervacije'].notna()]
data = data.reset_index(drop=True)
data.head(5)

,hotel_id,datum_dolaska,datum_kreiranja_rezervacije,datum_odjave,datum_otkazivanja_rezervacije,broj_odraslih_gostiju,broj_djece_gostiju,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,status_rezervacije,rezervacija_id,gost_id,ukupno_gostiju,duljina_boravka,raspon_dolazak_rezervacija
0,0,2015-07-01,2015-05-13,2015-07-02,None,1,0.0,GBR,0,1,64.991345,Check-Out,1313225,1039896,1,1,49
1,0,2015-07-01,2014-05-30,2015-07-02,None,1,0.0,GBR,1,1,74.368897,Check-Out,1313226,1008245,1,1,397
2,0,2015-07-01,2014-07-06,2015-07-03,None,2,0.0,GBR,2,1,130.973278,Check-Out,1313227,1093703,2,2,360
3,0,2015-07-01,2015-04-21,2015-07-03,None,2,0.0,GBR,2,1,143.304560,Check-Out,1313228,1044263,2,2,71
4,0,2015-07-01,2015-06-25,2015-07-03,None,2,0.0,PRT,0,0,161.502814,Check-Out,1313229,1022260,2,2,6


In [12]:
data_canceled.head()

,hotel_id,datum_dolaska,datum_kreiranja_rezervacije,datum_odjave,datum_otkazivanja_rezervacije,broj_odraslih_gostiju,broj_djece_gostiju,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,status_rezervacije,rezervacija_id,gost_id,ukupno_gostiju,duljina_boravka,raspon_dolazak_rezervacija
0,0,2015-07-01,2015-06-30,2015-07-04,2015-05-06,2,0.0,PRT,2,1,61.437192,Canceled,1313231,1075633,2,3,1
1,0,2015-07-01,2015-06-28,2015-07-04,2015-04-22,2,0.0,PRT,2,2,150.365262,Canceled,1313232,1056139,2,3,3
2,0,2015-07-01,2015-04-17,2015-07-05,2015-06-23,2,0.0,PRT,2,3,142.635814,Canceled,1313233,1020092,2,4,75
3,0,2015-07-01,2015-01-04,2015-07-08,2015-05-11,2,0.0,PRT,2,3,113.277032,Canceled,1313250,1018277,2,7,178
4,0,2015-07-01,2015-01-01,2015-07-11,2015-05-29,2,0.0,PRT,0,3,105.416363,Canceled,1313255,1035627,2,10,181


####  Uklanjanje stupaca: 
   - datum otkazivanja rezervacije (ovdje imamo samo neotkazane rezervacije)
   - datum odjave : jer je autokoreliran s stupcima datum dolaska i duljina boravka
   - broj_odraslih_gostiju	te broj_djece_gostiju uklanjamo kako smo dodali stupac ukupan broj gostiju
   - status rezervacije (sve su vrijednosti iste iste - 'Check-Out')
   - rezervacija_id - od 74553 zapisa imamo i 74553 unikatne vrijednosti identifikatora
    (NE ODBACUJEMO gost_id jer od 74553 zapisaimamo 69764 gosta, što znači da imamo ponavljajućih gostiju)
   - datum_kreiranja_rezervacije - dodan je stupac raspon_dolazak_rezervacija koji označava razliku dana dolaska i kreiranja rezervacije 
   

In [13]:
data = data.drop(columns = ['datum_otkazivanja_rezervacije','datum_odjave','broj_odraslih_gostiju','broj_djece_gostiju','status_rezervacije','rezervacija_id','datum_kreiranja_rezervacije'],axis=1)
data_canceled = data_canceled.drop(columns = ['datum_odjave','broj_odraslih_gostiju','broj_djece_gostiju','status_rezervacije','rezervacija_id','datum_kreiranja_rezervacije'],axis=1)
data.head()

,hotel_id,datum_dolaska,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,gost_id,ukupno_gostiju,duljina_boravka,raspon_dolazak_rezervacija
0,0,2015-07-01,GBR,0,1,64.991345,1039896,1,1,49
1,0,2015-07-01,GBR,1,1,74.368897,1008245,1,1,397
2,0,2015-07-01,GBR,2,1,130.973278,1093703,2,2,360
3,0,2015-07-01,GBR,2,1,143.304560,1044263,2,2,71
4,0,2015-07-01,PRT,0,0,161.502814,1022260,2,2,6


#### Odvajanje na hotel 0 (Resort) te na hotel 1 (City)

In [14]:
data_transformed = data.copy()
dataResort = data_transformed[data_transformed['hotel_id'] == 0]
dataCity = data_transformed[data_transformed['hotel_id'] == 1]

assert len(dataResort) + len(dataCity) == len(data_transformed)

# Stvaranje podatkovnih okvira na dnevnoj, tjednoj, mjesečnoj i godišnjoj razini za oba hotela

## Hotel 1 (City Hotel)

### Dnevna razina

In [19]:
daily_guest_data_city_hotel = {}

for index, row in dataCity.iterrows():
    arrival_date = row['datum_dolaska']
    length_of_stay = row['duljina_boravka']
    number_of_guests = row['ukupno_gostiju']
    total_price = row['cijena_nocenja']

    price_per_person_per_night = total_price / number_of_guests

    for day in range(length_of_stay + 1):
        current_day = arrival_date + timedelta(days=day)

        if current_day not in daily_guest_data_city_hotel:
            daily_guest_data_city_hotel[current_day] = {'ukupno_gostiju': 0, 'cijene_po_osobi': []}

        daily_guest_data_city_hotel[current_day]['ukupno_gostiju'] += number_of_guests
        daily_guest_data_city_hotel[current_day]['cijene_po_osobi'].extend([price_per_person_per_night] * number_of_guests)

for date, data in daily_guest_data_city_hotel.items():
    data['prosjecna_cijena_po_osobi'] = sum(data['cijene_po_osobi']) / len(data['cijene_po_osobi']) if data['cijene_po_osobi'] else 0

In [20]:
daily_guest_prices_data = [(date, data['ukupno_gostiju'], data['prosjecna_cijena_po_osobi']) for date, data in daily_guest_data_city_hotel.items()]
NumberOfGuestsDailyCity = pd.DataFrame(daily_guest_prices_data, columns=['datum', 'ukupno_gostiju', 'prosjecna_cijena_po_osobi'])
NumberOfGuestsDailyCity['datum'] = pd.to_datetime(NumberOfGuestsDailyCity['datum'])
NumberOfGuestsDailyCity = NumberOfGuestsDailyCity.sort_values(by='datum')
NumberOfGuestsDailyCity = NumberOfGuestsDailyCity.set_index('datum')


In [21]:
min_date = NumberOfGuestsDailyCity.index.min()
max_date = NumberOfGuestsDailyCity.index.max()
all_dates = pd.date_range(start=min_date, end=max_date, freq='D')

In [22]:
NumberOfGuestsDailyCity = NumberOfGuestsDailyCity.reindex(all_dates, fill_value=0)

In [23]:
NumberOfGuestsDailyCity.head(5)

,ukupno_gostiju,prosjecna_cijena_po_osobi
2015-01-01,10,86.098601
2015-01-02,16,80.030902
2015-01-03,27,68.309416
2015-01-04,27,67.628558
2015-01-05,28,66.300359


In [25]:
daily_prices_per_person = {}

for index, row in dataCity.iterrows():
    arrival_date = row['datum_dolaska']
    number_of_guests = row['ukupno_gostiju']
    total_price = row['cijena_nocenja']
    price_per_person = total_price / number_of_guests
    if arrival_date not in daily_prices_per_person:
        daily_prices_per_person[arrival_date] = []
    daily_prices_per_person[arrival_date].extend([price_per_person] * number_of_guests)

daily_mean_prices = {date: sum(prices) / len(prices) if prices else 0 for date, prices in daily_prices_per_person.items()}

daily_mean_prices_data = pd.DataFrame([(date, mean_price) for date, mean_price in daily_mean_prices.items()], columns=['datum_dolaska', 'prosjecna_cijena_po_osobi'])

NumberOfArrivalsDailyCity = dataCity.groupby('datum_dolaska').size().to_frame(name='broj_dolazaka')
NumberOfArrivalsDailyCity = NumberOfArrivalsDailyCity.merge(daily_mean_prices_data, on='datum_dolaska', how='left')
NumberOfArrivalsDailyCity.set_index('datum_dolaska', inplace=True)
NumberOfArrivalsDailyCity = NumberOfArrivalsDailyCity.resample('D').asfreq().fillna({'broj_dolazaka': 0, 'prosjecna_cijena_po_osobi': 0})
NumberOfArrivalsDailyCity['broj_dolazaka'] = NumberOfArrivalsDailyCity['broj_dolazaka'].astype(int)


In [26]:
NumberOfArrivalsDailyCity.head(5)

,broj_dolazaka,prosjecna_cijena_po_osobi
datum_dolaska,,
2015-01-01,6,86.098601
2015-01-02,3,69.918070
2015-01-03,5,51.259983
2015-01-04,5,69.173692
2015-01-05,2,75.949916


In [28]:
dataCity['datum_dolaska'] = pd.to_datetime(dataCity['datum_dolaska'])
NumberOfNewGuestsDailyCity = dataCity['ukupno_gostiju'].groupby(dataCity['datum_dolaska']).sum().to_frame(name='ukupno_gostiju')
NumberOfNewGuestsDailyCity['prosjecna_cijena_po_osobi'] = NumberOfArrivalsDailyCity['prosjecna_cijena_po_osobi']
NumberOfNewGuestsDailyCity = NumberOfNewGuestsDailyCity.resample('D').asfreq().fillna({'ukupno_gostiju': 0, 'prosjecna_cijena_po_osobi': 0})
NumberOfNewGuestsDailyCity['ukupno_gostiju'] = NumberOfNewGuestsDailyCity['ukupno_gostiju'].astype(int)

C:\Temp\ipykernel_24632\2951644865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataCity['datum_dolaska'] = pd.to_datetime(dataCity['datum_dolaska'])


In [45]:
NumberOfNewGuestsDailyCity.tail()

,ukupno_gostiju,prosjecna_cijena_po_osobi
datum_dolaska,,
2017-12-27,161,97.279942
2017-12-28,130,106.187584
2017-12-29,152,96.951495
2017-12-30,124,94.062967
2017-12-31,69,109.207817


### Tjedna razina

KOMENTAR 
    - modeliranje broja gostiju na tjednoj, mjesečnoj i godišnjoj bazi ima sljedeće probleme:
        - uzmemo li na primjer dvije osobe koje borave u istom hotelu u istom tjednu, gledanjem broja gostiju na                tjednoj bazi zanemarujemo razliku u slučajevima: osobe borave na različite dane u tjednu u hotelu (ukupan            broj gostiju +2), osobe borave na identične dane u tjednu u hotelu (opet ukupan broj +2) te osobe samo               djelomično borave na iste dane u hotelu (ukupan broj +2)
        - prethodno + osoba boravi u hotelu tokom dva ili više tjedana

Trenutni kod prolazi kroz svaku rezervaciju te gleda vrijeme dolaska i odlaska te se broj gostiju uvećava za broj gostiju rezervacije za tjedan/mjesec dolaska, boravka ili odlaksa 
Time dobivamo informaciju o broju gostiju u nekom tjednu/mjesecu u apsolutnom iznosu, odnosno ukupan broj gostiju u tom periodu. Isto tako imamo i podatkovni okvir koji čuva infomraciju o broju "novih" gostiju, odnosno promatra se samo dan/tjedan/mjesec dolaska te se na taj način modelira broj doalazaka u nekom vremenskom intervalu (tu se s druge strane gubi informacija o duljini boravka gostiju).
Moguće je rekreirat maximalnu dnevnu okupiranost na tjednoj i mjesečnoj razini (maksimalni dnevni iznos broja gostiju u tjednu/mjesecu) te na taj način modelirati broj gostiju na tjednoj i mjesečnoj razini.
        
        

In [72]:
weekly_guest_counts_city_hotel = {}

for index, row in dataCity.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']
    total_price = row['cijena_nocenja']
    price_per_person_per_night = total_price / number_of_guests

    arrival_week_start = arrival_date - timedelta(days=arrival_date.weekday())
    if arrival_week_start.year == 2014:
        arrival_week_start = pd.Timestamp(year=2015, month=1, day=1)

    departure_week_start = departure_date - timedelta(days=departure_date.weekday())
    
    if departure_week_start.year == 2014:
        departure_week_start = pd.Timestamp(year=2015, month=1, day=1)

    current_week_start = arrival_week_start
    while current_week_start <= departure_week_start:
        if current_week_start not in weekly_guest_counts_city_hotel:
            weekly_guest_counts_city_hotel[current_week_start] = {'ukupno_gostiju': 0, 'cijene_po_osobi': []}

        weekly_guest_counts_city_hotel[current_week_start]['ukupno_gostiju'] += number_of_guests
        weekly_guest_counts_city_hotel[current_week_start]['cijene_po_osobi'].extend([price_per_person_per_night] * number_of_guests)

        current_week_start += timedelta(days=7)

for date, data in weekly_guest_counts_city_hotel.items():
    data['prosjecna_cijena_po_osobi'] = sum(data['cijene_po_osobi']) / len(data['cijene_po_osobi']) if data['cijene_po_osobi'] else 0

weekly_guest_prices_data = [(date, data['ukupno_gostiju'], data['prosjecna_cijena_po_osobi']) for date, data in weekly_guest_counts_city_hotel.items()]
NumberOfGuestsWeeklyCity = pd.DataFrame(weekly_guest_prices_data, columns=['datum_dolaska', 'ukupno_gostiju', 'prosjecna_cijena_po_osobi'])


In [73]:
NumberOfGuestsWeeklyCity['datum_dolaska'] = pd.to_datetime(NumberOfGuestsWeeklyCity['datum_dolaska'])
NumberOfGuestsWeeklyCity = NumberOfGuestsWeeklyCity.sort_values(by='datum_dolaska')
NumberOfGuestsWeeklyCity = NumberOfGuestsWeeklyCity.set_index('datum_dolaska')


start_date = pd.Timestamp('2015-01-05')
end_date = pd.Timestamp('2018-01-08')
all_weekly_dates = pd.date_range(start=start_date, end=end_date, freq='W-MON')

earliest_date = NumberOfGuestsWeeklyCity.index.min()
if earliest_date < start_date:
    all_weekly_dates = all_weekly_dates.insert(0, earliest_date)

NumberOfGuestsWeeklyCity = NumberOfGuestsWeeklyCity.reindex(all_weekly_dates, fill_value=0)



,ukupno_gostiju,prosjecna_cijena_po_osobi
2015-01-01,35,68.506965
2015-01-05,42,58.668540
2015-01-12,70,63.990079
2015-01-19,78,72.850470
2015-01-26,52,69.177032


In [75]:
NumberOfGuestsWeeklyCity.head()

,ukupno_gostiju,prosjecna_cijena_po_osobi
2015-01-01,35,68.506965
2015-01-05,42,58.668540
2015-01-12,70,63.990079
2015-01-19,78,72.850470
2015-01-26,52,69.177032


In [39]:
if NumberOfArrivalsDailyCity.index.min() == pd.Timestamp('2015-01-01'):
    week_start_offset = pd.DateOffset(days=6)
else:
    week_start_offset = pd.DateOffset()

weekly_arrivals = NumberOfArrivalsDailyCity['broj_dolazaka'].resample('W').sum()
weekly_arrivals.index = weekly_arrivals.index - week_start_offset

weekly_average_price = NumberOfArrivalsDailyCity['prosjecna_cijena_po_osobi'].resample('W').mean()
weekly_average_price.index = weekly_average_price.index - week_start_offset

NumberOfArrivalsWeeklyCity = pd.DataFrame({'broj_dolazaka': weekly_arrivals, 'prosjecna_cijena_po_tjednu': weekly_average_price})

NumberOfArrivalsWeeklyCity.reset_index(inplace=True)
NumberOfArrivalsWeeklyCity['datum_dolaska'] = NumberOfArrivalsWeeklyCity['datum_dolaska'].apply(lambda x: pd.Timestamp('2015-01-01') if x.year == 2014 else x)
NumberOfArrivalsWeeklyCity.set_index('datum_dolaska', inplace=True)

In [40]:
NumberOfArrivalsWeeklyCity.head()

,broj_dolazaka,prosjecna_cijena_po_tjednu
datum_dolaska,,
2015-01-01,19,69.112586
2015-01-05,23,60.484136
2015-01-12,33,60.620359
2015-01-19,32,66.748246
2015-01-26,15,53.939380


In [41]:
if NumberOfNewGuestsDailyCity.index.min() == pd.Timestamp('2015-01-01'):
    week_start_offset = pd.DateOffset(days=6)
else:
    week_start_offset = pd.DateOffset()

weekly_new_guests = NumberOfNewGuestsDailyCity['ukupno_gostiju'].resample('W').sum()
weekly_new_guests.index = weekly_new_guests.index - week_start_offset

weekly_average_price = NumberOfNewGuestsDailyCity['prosjecna_cijena_po_osobi'].resample('W').mean()
weekly_average_price.index = weekly_average_price.index - week_start_offset

NumberOfNewGuestsWeeklyCity = pd.DataFrame({'ukupno_gostiju': weekly_new_guests, 'prosjecna_cijena_po_tjednu': weekly_average_price})
NumberOfNewGuestsWeeklyCity.reset_index(inplace=True)
NumberOfNewGuestsWeeklyCity['datum_dolaska'] = NumberOfNewGuestsWeeklyCity['datum_dolaska'].apply(lambda x: pd.Timestamp('2015-01-01') if x.year == 2014 else x)
NumberOfNewGuestsWeeklyCity.set_index('datum_dolaska', inplace=True)

In [42]:
NumberOfNewGuestsWeeklyCity.head()

,ukupno_gostiju,prosjecna_cijena_po_tjednu
datum_dolaska,,
2015-01-01,35,69.112586
2015-01-05,42,60.484136
2015-01-12,54,60.620359
2015-01-19,48,66.748246
2015-01-26,27,53.939380


### Mjesečna razina

In [79]:
monthly_guest_counts_city_hotel = {}

for index, row in dataCity.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']
    total_price = row['cijena_nocenja']
    price_per_person_per_night = total_price / number_of_guests

    current_month_start = arrival_date.replace(day=1)
    while current_month_start <= departure_date.replace(day=1):
        if current_month_start not in monthly_guest_counts_city_hotel:
            monthly_guest_counts_city_hotel[current_month_start] = {'ukupno_gostiju': 0, 'cijene_po_osobi': []}

        monthly_guest_counts_city_hotel[current_month_start]['ukupno_gostiju'] += number_of_guests
        monthly_guest_counts_city_hotel[current_month_start]['cijene_po_osobi'].extend([price_per_person_per_night] * number_of_guests)
        if current_month_start.month == 12: 
            current_month_start = current_month_start.replace(year=current_month_start.year + 1, month=1, day=1)
        else:
            current_month_start = current_month_start.replace(month=current_month_start.month + 1, day=1)

for date, data in monthly_guest_counts_city_hotel.items():
    data['prosjecna_cijena_po_osobi'] = sum(data['cijene_po_osobi']) / len(data['cijene_po_osobi']) if data['cijene_po_osobi'] else 0

In [80]:
monthly_guest_prices_data_city = [(date, data['ukupno_gostiju'], data['prosjecna_cijena_po_osobi']) for date, data in monthly_guest_counts_city_hotel.items()]
NumberOfGuestsMonthlyCity = pd.DataFrame(monthly_guest_prices_data_city, columns=['datum_dolaska', 'ukupno_gostiju', 'prosjecna_cijena_po_osobi'])
NumberOfGuestsMonthlyCity = NumberOfGuestsMonthlyCity.sort_values(by='datum_dolaska')
NumberOfGuestsMonthlyCity = NumberOfGuestsMonthlyCity.set_index('datum_dolaska')

In [94]:
departure_dates = dataCity['datum_dolaska'] + pd.to_timedelta(dataCity['duljina_boravka'], unit='D')

latest_departure_date = departure_dates.max()

earliest_date = dataCity['datum_dolaska'].min()
all_months = pd.date_range(start=earliest_date, end=latest_departure_date, freq='MS').to_series()
all_months_df = pd.DataFrame(all_months, columns=['datum_dolaska'])
all_months_df.set_index('datum_dolaska', inplace=True)

NumberOfGuestsMonthlyCity = all_months_df.join(NumberOfGuestsMonthlyCity, how='left').fillna({'ukupno_gostiju': 0, 'prosjecna_cijena_po_osobi': 0})

In [95]:
NumberOfGuestsMonthlyCity.head()

,ukupno_gostiju,prosjecna_cijena_po_osobi
datum_dolaska,,
2015-01-01,206.0,66.099957
2015-02-01,9.0,68.238280
2015-03-01,0.0,0.000000
2015-04-01,0.0,0.000000
2015-05-01,1067.0,48.706533


In [96]:
NumberOfArrivalsMonthlyCity = NumberOfArrivalsDailyCity['broj_dolazaka'].resample('MS').sum()
monthly_average_price = NumberOfArrivalsDailyCity['prosjecna_cijena_po_osobi'].resample('MS').mean()
NumberOfArrivalsMonthlyCity = pd.DataFrame({'broj_dolazaka': NumberOfArrivalsMonthlyCity, 'prosjecna_cijena_po_mjesecu': monthly_average_price})

In [97]:
NumberOfArrivalsMonthlyCity.head()

,broj_dolazaka,prosjecna_cijena_po_mjesecu
datum_dolaska,,
2015-01-01,122,63.515974
2015-02-01,0,0.000000
2015-03-01,0,0.000000
2015-04-01,0,0.000000
2015-05-01,574,49.893909


In [98]:
NumberOfNewGuestsMonthlyCity = NumberOfNewGuestsDailyCity['ukupno_gostiju'].resample('MS').sum()
monthly_average_price = NumberOfNewGuestsDailyCity['prosjecna_cijena_po_osobi'].resample('MS').mean()
NumberOfNewGuestsMonthlyCity = pd.DataFrame({'ukupno_gostiju': NumberOfNewGuestsMonthlyCity, 'prosjecna_cijena_po_mjesecu': monthly_average_price})

In [99]:
NumberOfNewGuestsMonthlyCity.head()

,ukupno_gostiju,prosjecna_cijena_po_mjesecu
datum_dolaska,,
2015-01-01,206,63.515974
2015-02-01,0,0.000000
2015-03-01,0,0.000000
2015-04-01,0,0.000000
2015-05-01,1067,49.893909


### Godišnja razina

In [100]:
yearly_guest_counts_city_hotel = {}

for index, row in dataCity.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']
    total_price = row['cijena_nocenja']
    price_per_person_per_night = total_price / number_of_guests

    current_year_start = arrival_date.replace(month=1, day=1)

    while current_year_start.year <= departure_date.year:
        if current_year_start not in yearly_guest_counts_city_hotel:
            yearly_guest_counts_city_hotel[current_year_start] = {'ukupno_gostiju': 0, 'cijene_po_osobi': []}

        yearly_guest_counts_city_hotel[current_year_start]['ukupno_gostiju'] += number_of_guests
        yearly_guest_counts_city_hotel[current_year_start]['cijene_po_osobi'].extend([price_per_person_per_night] * number_of_guests)

        current_year_start = current_year_start.replace(year=current_year_start.year + 1, month=1, day=1)
        
for date, data in yearly_guest_counts_city_hotel.items():
    data['prosjecna_cijena_po_osobi'] = sum(data['cijene_po_osobi']) / len(data['cijene_po_osobi']) if data['cijene_po_osobi'] else 0


In [103]:
yearly_guest_prices_data = [(date, data['ukupno_gostiju'], data['prosjecna_cijena_po_osobi']) for date, data in yearly_guest_counts_city_hotel.items()]
NumberOfGuestsYearlyCity = pd.DataFrame(yearly_guest_prices_data, columns=['datum_dolaska', 'ukupno_gostiju', 'prosjecna_cijena_po_osobi'])
NumberOfGuestsYearlyCity = NumberOfGuestsYearlyCity.sort_values(by='datum_dolaska')
NumberOfGuestsYearlyCity = NumberOfGuestsYearlyCity.set_index('datum_dolaska')

In [104]:
NumberOfGuestsYearlyCity = NumberOfGuestsYearlyCity.drop(pd.Timestamp('2018-01-01'))


In [105]:
NumberOfGuestsYearlyCity.head()

,ukupno_gostiju,prosjecna_cijena_po_osobi
datum_dolaska,,
2015-01-01,13642,60.073296
2016-01-01,44621,62.249559
2017-01-01,32100,66.335675


In [106]:
yearly_arrivals = NumberOfArrivalsDailyCity['broj_dolazaka'].resample('YS').sum()
yearly_average_price = NumberOfArrivalsDailyCity['prosjecna_cijena_po_osobi'].resample('YS').mean()
NumberOfArrivalsYearlyCity = pd.DataFrame({'broj_dolazaka': yearly_arrivals, 'prosjecna_cijena_po_godini': yearly_average_price})

In [107]:
NumberOfArrivalsYearlyCity.head()

,broj_dolazaka,prosjecna_cijena_po_godini
datum_dolaska,,
2015-01-01,7598,37.690342
2016-01-01,22493,57.440850
2017-01-01,15705,59.625294


In [108]:
yearly_new_guests = NumberOfNewGuestsDailyCity['ukupno_gostiju'].resample('YS').sum()
yearly_average_price_new_guests = NumberOfNewGuestsDailyCity['prosjecna_cijena_po_osobi'].resample('YS').mean()
NumberOfNewGuestsYearlyCity = pd.DataFrame({'ukupno_gostiju': yearly_new_guests, 'prosjecna_cijena_po_godini': yearly_average_price_new_guests})

In [109]:
NumberOfNewGuestsYearlyCity.head()

,ukupno_gostiju,prosjecna_cijena_po_godini
datum_dolaska,,
2015-01-01,13642,37.690342
2016-01-01,44131,57.440850
2017-01-01,31170,59.625294


## Hotel 0 (Resort Hotel)

### Dnevna razina

In [75]:
daily_guest_counts_resort_hotel = {}

for index, row in dataResort.iterrows():
    arrival_date = row['datum_dolaska']
    length_of_stay = row['duljina_boravka']
    number_of_guests = row['ukupno_gostiju']
    total_price = row['cijena_nocenja']
    price_per_person_per_night = total_price / number_of_guests

    for day in range(length_of_stay + 1):
        current_day = arrival_date + timedelta(days=day)
        if current_day not in daily_guest_counts_resort_hotel:
            daily_guest_counts_resort_hotel[current_day] = {'ukupno_gostiju': 0, 'cijene_po_osobi': []}

        daily_guest_counts_resort_hotel[current_day]['ukupno_gostiju'] += number_of_guests
        daily_guest_counts_resort_hotel[current_day]['cijene_po_osobi'].extend([price_per_person_per_night] * number_of_guests)

for date, data in daily_guest_counts_resort_hotel.items():
    data['prosjecna_cijena_po_osobi'] = sum(data['cijene_po_osobi']) / len(data['cijene_po_osobi']) if data['cijene_po_osobi'] else 0

daily_guest_prices_data = [(date, data['ukupno_gostiju'], data['prosjecna_cijena_po_osobi']) for date, data in daily_guest_counts_resort_hotel.items()]
NumberOfGuestsDailyResort = pd.DataFrame(daily_guest_prices_data, columns=['datum', 'ukupno_gostiju', 'prosjecna_cijena_po_osobi'])
NumberOfGuestsDailyResort = NumberOfGuestsDailyResort.sort_values(by='datum').set_index('datum')


In [76]:
NumberOfGuestsDailyResort.head(5)

,ukupno_gostiju,prosjecna_cijena_po_osobi
datum,,
2015-07-01,63,57.719417
2015-07-02,135,63.494862
2015-07-03,187,62.056435
2015-07-04,245,58.090289
2015-07-05,307,57.317928


In [77]:
daily_prices_per_person_resort = {}

for index, row in dataResort.iterrows():
    arrival_date = row['datum_dolaska']
    number_of_guests = row['ukupno_gostiju']
    total_price = row['cijena_nocenja']
    price_per_person = total_price / number_of_guests
    if arrival_date not in daily_prices_per_person_resort :
        daily_prices_per_person_resort[arrival_date] = []
    daily_prices_per_person_resort[arrival_date].extend([price_per_person] * number_of_guests)

daily_mean_prices_resort = {date: sum(prices) / len(prices) if prices else 0 for date, prices in daily_prices_per_person_resort.items()}

daily_mean_prices_data_resort = pd.DataFrame([(date, mean_price) for date, mean_price in daily_mean_prices_resort.items()], columns=['datum_dolaska', 'prosjecna_cijena_po_osobi'])

NumberOfArrivalsDailyResort = dataResort.groupby('datum_dolaska').size().to_frame(name='broj_dolazaka')
NumberOfArrivalsDailyResort = NumberOfArrivalsDailyResort.merge(daily_mean_prices_data_resort , on='datum_dolaska', how='left')
NumberOfArrivalsDailyResort.set_index('datum_dolaska', inplace=True)
NumberOfArrivalsDailyResort = NumberOfArrivalsDailyResort.resample('D').asfreq().fillna({'broj_dolazaka': 0, 'prosjecna_cijena_po_osobi': 0})
NumberOfArrivalsDailyResort['broj_dolazaka'] = NumberOfArrivalsDailyResort['broj_dolazaka'].astype(int)


In [78]:
NumberOfArrivalsDailyResort.head(5)

,broj_dolazaka,prosjecna_cijena_po_osobi
datum_dolaska,,
2015-07-01,32,57.719417
2015-07-02,35,68.548378
2015-07-03,27,59.951927
2015-07-04,36,54.928429
2015-07-05,37,59.883143


In [79]:
NumberOfNewGuestsDailyResort = dataResort['ukupno_gostiju'].groupby(dataResort['datum_dolaska']).sum()
NumberOfNewGuestsDailyResort = NumberOfNewGuestsDailyResort.resample('d').sum().to_frame()
prices = NumberOfArrivalsDailyResort['prosjecna_cijena_po_osobi']
NumberOfNewGuestsDailyResort['prosjecna_cijena_po_osobi'] = prices

In [80]:
NumberOfNewGuestsDailyResort.head(5)

,ukupno_gostiju,prosjecna_cijena_po_osobi
datum_dolaska,,
2015-07-01,63,57.719417
2015-07-02,72,68.548378
2015-07-03,59,59.951927
2015-07-04,80,54.928429
2015-07-05,80,59.883143


### Tjedna razina

In [81]:
weekly_guest_counts_resort_hotel = {}

for index, row in dataResort.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']
    total_price = row['cijena_nocenja']
    price_per_person_per_night = total_price / number_of_guests

    arrival_week_start = arrival_date - timedelta(days=arrival_date.weekday())

    departure_week_start = departure_date - timedelta(days=departure_date.weekday())

    current_week_start = arrival_week_start
    while current_week_start <= departure_week_start:
        if current_week_start not in weekly_guest_counts_resort_hotel:
            weekly_guest_counts_resort_hotel[current_week_start] = {'ukupno_gostiju': 0, 'cijene_po_osobi': []}

        weekly_guest_counts_resort_hotel[current_week_start]['ukupno_gostiju'] += number_of_guests
        weekly_guest_counts_resort_hotel[current_week_start]['cijene_po_osobi'].extend([price_per_person_per_night] * number_of_guests)

        current_week_start += timedelta(days=7)

for date, data in weekly_guest_counts_resort_hotel.items():
    data['prosjecna_cijena_po_osobi'] = sum(data['cijene_po_osobi']) / len(data['cijene_po_osobi']) if data['cijene_po_osobi'] else 0


In [82]:
weekly_guest_prices_data_resort = [(date, data['ukupno_gostiju'], data['prosjecna_cijena_po_osobi']) for date, data in weekly_guest_counts_resort_hotel.items()]
NumberOfGuestsWeeklyResort = pd.DataFrame(weekly_guest_prices_data_resort, columns=['datum_dolaska', 'ukupno_gostiju', 'prosjecna_cijena_po_osobi'])
NumberOfGuestsWeeklyResort ['datum_dolaska'] = pd.to_datetime(NumberOfGuestsWeeklyResort ['datum_dolaska'])
NumberOfGuestsWeeklyResort  = NumberOfGuestsWeeklyResort .sort_values(by='datum_dolaska')
NumberOfGuestsWeeklyResort  = NumberOfGuestsWeeklyResort .set_index('datum_dolaska')

In [83]:
NumberOfGuestsWeeklyResort.head()

,ukupno_gostiju,prosjecna_cijena_po_osobi
datum_dolaska,,
2015-06-29,354,60.152248
2015-07-06,736,62.000597
2015-07-13,918,70.789282
2015-07-20,895,75.749091
2015-07-27,933,78.082672


In [84]:
week_starts = NumberOfGuestsWeeklyResort.index
weekly_arrivals = []
for i in range(len(week_starts) - 1):
    week_start = week_starts[i]
    next_week_start = week_starts[i + 1]
    weekly_total = NumberOfArrivalsDailyResort[week_start:next_week_start - pd.DateOffset(days=1)]['broj_dolazaka'].sum()
    weekly_arrivals.append((week_start, weekly_total))

NumberOfArrivalsWeeklyResort = pd.DataFrame(weekly_arrivals, columns=['datum_dolaska', 'broj_dolazaka'])
NumberOfArrivalsWeeklyResort.set_index('datum_dolaska', inplace=True)

NumberOfArrivalsWeeklyResort['prosjecna_cijena_po_tjednu'] = NumberOfArrivalsWeeklyResort.index.map(
    lambda week_start: NumberOfArrivalsDailyResort.loc[week_start:week_start + pd.DateOffset(days=6)]['prosjecna_cijena_po_osobi'].mean()
)

In [85]:
NumberOfArrivalsWeeklyResort.head()

,broj_dolazaka,prosjecna_cijena_po_tjednu
datum_dolaska,,
2015-06-29,167,60.206259
2015-07-06,215,64.792053
2015-07-13,249,76.307298
2015-07-20,231,75.242909
2015-07-27,245,78.464839


In [86]:
week_starts = NumberOfGuestsWeeklyResort.index.tolist()
weekly_new_guests_data = []

for i in range(len(week_starts) - 1):
    week_start = week_starts[i]
    next_week_start = week_starts[i + 1]

    weekly_total = NumberOfNewGuestsDailyResort[week_start:next_week_start - pd.Timedelta(days=1)]['ukupno_gostiju'].sum()
    
    weekly_new_guests_data.append([week_start, weekly_total])

NumberOfNewGuestsWeeklyResort = pd.DataFrame(weekly_new_guests_data, columns=['datum_dolaska', 'ukupno_gostiju'])
NumberOfNewGuestsWeeklyResort['ukupno_gostiju'] = NumberOfNewGuestsWeeklyResort['ukupno_gostiju'].astype(int) 
NumberOfNewGuestsWeeklyResort.set_index('datum_dolaska', inplace=True)

NumberOfNewGuestsWeeklyResort['prosjecna_cijena_po_tjednu'] = NumberOfNewGuestsWeeklyResort.index.map(
    lambda week_start: NumberOfNewGuestsDailyResort.loc[week_start:week_start + pd.Timedelta(days=6)]['prosjecna_cijena_po_osobi'].mean()
)


In [87]:
NumberOfNewGuestsWeeklyResort.head()

,ukupno_gostiju,prosjecna_cijena_po_tjednu
datum_dolaska,,
2015-06-29,354,60.206259
2015-07-06,483,64.792053
2015-07-13,535,76.307298
2015-07-20,521,75.242909
2015-07-27,547,78.464839


### Mjesečna razina

In [88]:
monthly_guest_counts_resort_hotel = {}

for index, row in dataResort.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']
    total_price = row['cijena_nocenja']
    price_per_person_per_night = total_price / number_of_guests

    current_month_start = arrival_date.replace(day=1)
    while current_month_start <= departure_date.replace(day=1):
        if current_month_start not in monthly_guest_counts_resort_hotel:
            monthly_guest_counts_resort_hotel[current_month_start] = {'ukupno_gostiju': 0, 'cijene_po_osobi': []}

        monthly_guest_counts_resort_hotel[current_month_start]['ukupno_gostiju'] += number_of_guests
        monthly_guest_counts_resort_hotel[current_month_start]['cijene_po_osobi'].extend([price_per_person_per_night] * number_of_guests)

        if current_month_start.month == 12: 
            current_month_start = current_month_start.replace(year=current_month_start.year + 1, month=1, day=1)
        else:
            current_month_start = current_month_start.replace(month=current_month_start.month + 1, day=1)

for date, data in monthly_guest_counts_resort_hotel.items():
    data['prosjecna_cijena_po_osobi'] = sum(data['cijene_po_osobi']) / len(data['cijene_po_osobi']) if data['cijene_po_osobi'] else 0

monthly_guest_prices_data_resort = [(date, data['ukupno_gostiju'], data['prosjecna_cijena_po_osobi']) for date, data in monthly_guest_counts_resort_hotel.items()]
NumberOfGuestsMonthlyResort = pd.DataFrame(monthly_guest_prices_data_resort, columns=['datum_dolaska', 'ukupno_gostiju', 'prosjecna_cijena_po_osobi'])
NumberOfGuestsMonthlyResort = NumberOfGuestsMonthlyResort.sort_values(by='datum_dolaska')
NumberOfGuestsMonthlyResort = NumberOfGuestsMonthlyResort.set_index('datum_dolaska')

In [89]:
NumberOfGuestsMonthlyResort.head(5)

,ukupno_gostiju,prosjecna_cijena_po_osobi
datum_dolaska,,
2015-07-01,2263,71.583211
2015-08-01,2678,81.642151
2015-09-01,2362,60.777244
2015-10-01,2419,47.208524
2015-11-01,1722,50.238473


In [90]:
NumberOfArrivalsMonthlyResort = NumberOfArrivalsDailyResort['broj_dolazaka'].resample('MS').sum()
monthly_average_price = NumberOfArrivalsDailyResort['prosjecna_cijena_po_osobi'].resample('MS').mean()
NumberOfArrivalsMonthlyResort = pd.DataFrame({'broj_dolazaka': NumberOfArrivalsMonthlyResort, 'prosjecna_cijena_po_mjesecu': monthly_average_price})

In [91]:
NumberOfArrivalsMonthlyResort.head()

,broj_dolazaka,prosjecna_cijena_po_mjesecu
datum_dolaska,,
2015-07-01,1028,70.894378
2015-08-01,1014,82.154563
2015-09-01,1017,59.260092
2015-10-01,1115,47.276024
2015-11-01,884,51.774874


In [92]:
NumberOfNewGuestsMonthlyResort = NumberOfNewGuestsDailyResort['ukupno_gostiju'].resample('MS').sum()
monthly_average_price = NumberOfNewGuestsDailyResort['prosjecna_cijena_po_osobi'].resample('MS').mean()
NumberOfNewGuestsMonthlyResort = pd.DataFrame({'ukupno_gostiju': NumberOfNewGuestsMonthlyResort, 'prosjecna_cijena_po_mjesecu': monthly_average_price})

In [93]:
NumberOfNewGuestsMonthlyResort.head()

,ukupno_gostiju,prosjecna_cijena_po_mjesecu
datum_dolaska,,
2015-07-01,2263,70.894378
2015-08-01,2278,82.154563
2015-09-01,2006,59.260092
2015-10-01,2072,47.276024
2015-11-01,1501,51.774874


### Godišnja razina

In [94]:
yearly_guest_counts_resort_hotel = {}

for index, row in dataResort.iterrows():
    arrival_date = row['datum_dolaska']
    departure_date = arrival_date + pd.Timedelta(days=row['duljina_boravka'])
    number_of_guests = row['ukupno_gostiju']
    total_price = row['cijena_nocenja']
    price_per_person_per_night = total_price / number_of_guests

    current_year_start = arrival_date.replace(month=1, day=1)
    while current_year_start.year <= departure_date.year:
        if current_year_start not in yearly_guest_counts_resort_hotel:
            yearly_guest_counts_resort_hotel[current_year_start] = {'ukupno_gostiju': 0, 'cijene_po_osobi': []}

        yearly_guest_counts_resort_hotel[current_year_start]['ukupno_gostiju'] += number_of_guests
        yearly_guest_counts_resort_hotel[current_year_start]['cijene_po_osobi'].append(price_per_person_per_night)

        current_year_start = current_year_start.replace(year=current_year_start.year + 1, month=1, day=1)

for date, data in yearly_guest_counts_resort_hotel.items():
    data['prosjecna_cijena_po_osobi'] = sum(data['cijene_po_osobi']) / len(data['cijene_po_osobi']) if data['cijene_po_osobi'] else 0

yearly_guest_data_resort = [(date, data['ukupno_gostiju'], data['prosjecna_cijena_po_osobi']) for date, data in yearly_guest_counts_resort_hotel.items()]
NumberOfGuestsYearlyResort = pd.DataFrame(yearly_guest_data_resort, columns=['datum_dolaska', 'ukupno_gostiju', 'prosjecna_cijena_po_osobi'])
NumberOfGuestsYearlyResort = NumberOfGuestsYearlyResort.sort_values(by='datum_dolaska')
NumberOfGuestsYearlyResort = NumberOfGuestsYearlyResort.set_index('datum_dolaska')


In [95]:
NumberOfGuestsYearlyResort.head()

,ukupno_gostiju,prosjecna_cijena_po_osobi
datum_dolaska,,
2015-01-01,11858,65.640037
2016-01-01,26068,64.055880
2017-01-01,18056,71.256832


In [96]:
yearly_arrivals = NumberOfArrivalsDailyResort['broj_dolazaka'].resample('YS').sum()
yearly_average_price = NumberOfArrivalsDailyResort['prosjecna_cijena_po_osobi'].resample('YS').mean()
NumberOfArrivalsYearlyResort = pd.DataFrame({'broj_dolazaka': yearly_arrivals, 'prosjecna_cijena_po_godini': yearly_average_price})

In [97]:
NumberOfArrivalsYearlyResort

,broj_dolazaka,prosjecna_cijena_po_godini
datum_dolaska,,
2015-01-01,5970,60.754182
2016-01-01,13183,59.684018
2017-01-01,8987,67.012692


In [98]:
yearly_new_guests = NumberOfNewGuestsDailyResort['ukupno_gostiju'].resample('YS').sum()
yearly_average_price_new_guests = NumberOfNewGuestsDailyResort['prosjecna_cijena_po_osobi'].resample('YS').mean()
NumberOfNewGuestsYearlyResort = pd.DataFrame({'ukupno_gostiju': yearly_new_guests, 'prosjecna_cijena_po_godini': yearly_average_price_new_guests})

In [99]:
NumberOfNewGuestsYearlyResort

,ukupno_gostiju,prosjecna_cijena_po_godini
datum_dolaska,,
2015-01-01,11858,60.754182
2016-01-01,25708,59.684018
2017-01-01,17715,67.012692


# Spremanje podatakovnih okvira (interim za sada)

In [100]:
DATA_PATH = '../data/interim'
DATA_HOTEL0_BASE_PATH = '../data/interim/hotel_0'
DATA_HOTEL1_BASE_PATH = '../data/interim/hotel_1'

In [101]:
dataResort.head()

,hotel_id,datum_dolaska,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,gost_id,ukupno_gostiju,duljina_boravka,raspon_dolazak_rezervacija
0,0,2015-07-01,GBR,0,1,64.991345,1039896,1,1,49
1,0,2015-07-01,GBR,1,1,74.368897,1008245,1,1,397
2,0,2015-07-01,GBR,2,1,130.973278,1093703,2,2,360
3,0,2015-07-01,GBR,2,1,143.304560,1044263,2,2,71
4,0,2015-07-01,PRT,0,0,161.502814,1022260,2,2,6


In [102]:
dataCity.head()

,hotel_id,datum_dolaska,zemlja_gosta,kanal_prodaje_id,tip_sobe_id,cijena_nocenja,gost_id,ukupno_gostiju,duljina_boravka,raspon_dolazak_rezervacija
28140,1,2015-11-01,PRT,2,1,100.000000,1039201,1,2,1
28141,1,2015-07-02,PRT,2,1,58.274878,1037233,1,3,171
28142,1,2015-10-03,PRT,2,1,103.238057,1055376,2,2,247
28143,1,2015-07-03,PRT,2,1,133.579567,1045475,2,2,445
28144,1,2015-12-03,PRT,2,1,135.267131,1018997,2,2,157


In [103]:
data_transformed.to_parquet(os.path.join(DATA_PATH,"data_no_cancellations.parquet"))
data_canceled.to_parquet(os.path.join(DATA_PATH,"data_cancellations.parquet"))
dataResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"hotel0cleaned.parquet"))
dataCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"hotel1cleaned.parquet"))



In [104]:
NumberOfGuestsDailyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"daily","daily_guests.parquet"))
NumberOfArrivalsDailyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"daily","daily_arrivals.parquet"))
NumberOfNewGuestsDailyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"daily","daily_new_guests.parquet"))
NumberOfGuestsWeeklyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"weekly","weekly_guests.parquet"))
NumberOfArrivalsWeeklyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"weekly","weekly_arrivals.parquet"))
NumberOfNewGuestsWeeklyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"weekly","weekly_new_guests.parquet"))
NumberOfGuestsMonthlyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"monthly","monthly_guests.parquet"))
NumberOfArrivalsMonthlyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"monthly","monthly_arrivals.parquet"))
NumberOfNewGuestsMonthlyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"monthly","monthly_new_guests.parquet"))
NumberOfGuestsYearlyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"yearly","yearly_guests.parquet"))
NumberOfArrivalsYearlyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"yearly","yearly_arrivals.parquet"))
NumberOfNewGuestsYearlyResort.to_parquet(os.path.join(DATA_HOTEL0_BASE_PATH,"yearly","yearly_new_guests.parquet"))

In [105]:
NumberOfGuestsDailyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"daily","daily_guests.parquet"))
NumberOfArrivalsDailyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"daily","daily_arrivals.parquet")) 
NumberOfNewGuestsDailyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"daily","daily_new_guests.parquet"))
NumberOfGuestsWeeklyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"weekly","weekly_guests.parquet")) 
NumberOfArrivalsWeeklyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"weekly","weekly_arrivals.parquet")) 
NumberOfNewGuestsWeeklyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"weekly","weekly_new_guests.parquet"))
NumberOfGuestsMonthlyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"monthly","monthly_guests.parquet")) 
NumberOfArrivalsMonthlyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"monthly","monthly_arrivals.parquet"))
NumberOfNewGuestsMonthlyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"monthly","monthly_new_guests.parquet"))
NumberOfGuestsYearlyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"yearly","yearly_guests.parquet"))
NumberOfArrivalsYearlyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"yearly","yearly_arrivals.parquet")) 
NumberOfNewGuestsYearlyCity.to_parquet(os.path.join(DATA_HOTEL1_BASE_PATH,"yearly","yearly_new_guests.parquet"))